# Stage 03 part 1: Data cleaning and transformation

In this file we clean the data for a better future analysis. We divide this stage in steps:

1. Importing libraries
2. Loading the data
3. Data cleaning and transformation

### Step 1: importing libraries

In [33]:
import datetime 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)

### Step 2: loading the data


In [47]:
df = pd.read_parquet ('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2018-01.parquet')

### Step 3: data cleaning and transformation

#### a) Deleting innecesary columns

In [48]:
df.drop(columns=['congestion_surcharge','airport_fee'],inplace=True)

#### b) Changing negative values for positive values

In [49]:
df.fare_amount = df.fare_amount.abs()
df.extra = df.extra.abs()
df.mta_tax = df.mta_tax.abs()
df.tip_amount = df.tip_amount.abs()
df.tolls_amount = df.tolls_amount.abs()
df.improvement_surcharge = df.improvement_surcharge.abs()
df.total_amount = df.total_amount.abs()

#### c) Deleting duplicate values

In [5]:
df[df.duplicated()]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
75758,2,2018-01-01 05:52:21,2018-01-01 05:54:17,1,0.15,1,N,170,137,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8
424897,2,2018-01-02 19:07:46,2018-01-02 19:11:24,1,0.30,1,N,238,238,2,3.5,1.0,0.5,0.0,0.0,0.3,5.3
440994,2,2018-01-02 20:49:55,2018-01-02 20:50:06,1,0.02,1,N,186,186,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8
1500176,2,2018-01-07 12:16:16,2018-01-07 12:21:46,5,0.52,1,N,107,107,2,5.5,0.0,0.5,0.0,0.0,0.3,6.3
1743867,2,2018-01-08 12:35:07,2018-01-08 12:43:48,5,0.27,1,N,107,79,2,7.0,0.0,0.5,0.0,0.0,0.3,7.8
2396204,2,2018-01-10 18:39:24,2018-01-10 18:39:36,1,0.02,1,N,186,186,2,2.5,1.0,0.5,0.0,0.0,0.3,4.3
3861553,2,2018-01-15 16:15:02,2018-01-15 16:15:04,5,0.06,1,N,264,138,2,2.5,0.0,0.5,0.0,0.0,0.3,3.3
5237880,2,2018-01-20 01:48:34,2018-01-20 01:48:59,2,0.07,1,N,113,114,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8


In [50]:
df.drop_duplicates(inplace=True)

#### d) Deleting outliers

In [7]:
df[df.trip_distance > 500]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1858262,1,2018-01-08 19:44:54,2018-01-08 19:50:00,0,830.80,1,N,100,230,1,5.5,1.0,0.5,1.8,0.0,0.3,9.1
8237763,2,2018-01-30 11:41:02,2018-01-30 11:42:09,1,189483.84,1,N,193,193,2,4.0,0.0,0.0,0.0,0.0,0.0,4.0


In [51]:
df.drop([1858262,8237763],inplace=True)

#### e) We created a new column, fare_per_mile, to study the relation between fare_amount and trip_distance

In [52]:
df['fare_per_mile'] = df.fare_amount / df.trip_distance

There are fields with trip_distance 0. So fare_per_mile is infinite in these cases. We assigned the value of 0 to these fields.

In [53]:
df.fare_per_mile[df.fare_per_mile > 100000].unique()

array([inf])

In [54]:
df[df.fare_per_mile.isna()].count()

VendorID                 1163
tpep_pickup_datetime     1163
tpep_dropoff_datetime    1163
passenger_count          1163
trip_distance            1163
RatecodeID               1163
store_and_fwd_flag       1163
PULocationID             1163
DOLocationID             1163
payment_type             1163
fare_amount              1163
extra                    1163
mta_tax                  1163
tip_amount               1163
tolls_amount             1163
improvement_surcharge    1163
total_amount             1163
fare_per_mile               0
dtype: int64

In [55]:
df.fare_per_mile[df.fare_per_mile > 100000] = 0

C:\Users\urbda\AppData\Local\Temp\ipykernel_3080\956704063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fare_per_mile[df.fare_per_mile > 100000] = 0


In [56]:
df.fare_per_mile.fillna(0,inplace=True)

#### f) We created a new column, trip_time, for identify the trip time in seconds.

First, we calculate the time difference

In [57]:
df['trip_time'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

Then, we converted it to seconds

In [58]:
df.trip_time = df.trip_time.dt.total_seconds()

In [67]:
df[df.trip_time < 0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,fare_per_mile,trip_time,fare_per_minute
158888,1,2018-01-01 15:15:13,2017-12-28 16:03:38,1,3.3,1,N,209,137,1,13.0,0.5,0.5,2.85,0.00,0.3,17.15,3.939394,-342695.0,-0.002276
6146145,1,2018-01-23 13:12:19,2018-01-23 00:28:25,2,20.9,2,N,132,244,2,52.0,0.0,0.5,0.00,5.76,0.3,58.56,2.488038,-45834.0,-0.068072


In [68]:
df.drop([158888,6146145],inplace=True)

#### g) We created a new column, fare_per_minute, for indetify the relation between fare_amount and el trip_time

In [59]:
df['fare_per_minute'] = df.fare_amount / (df.trip_time / 60)

In [69]:
df.fare_per_minute.describe()

count    8.760675e+06
mean     2.488803e+00
std      4.991993e+01
min      0.000000e+00
25%      7.850467e-01
50%      9.128631e-01
75%      1.102941e+00
max      3.816333e+04
Name: fare_per_minute, dtype: float64

In [61]:
df.fare_per_minute[df.fare_per_minute > 100000].unique()

array([inf])

In [62]:
df.fare_per_minute[df.fare_per_minute > 100000] = 0

C:\Users\urbda\AppData\Local\Temp\ipykernel_3080\2332422894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fare_per_minute[df.fare_per_minute > 100000] = 0


In [63]:
df[df.fare_per_minute.isna()].count()

VendorID                 323
tpep_pickup_datetime     323
tpep_dropoff_datetime    323
passenger_count          323
trip_distance            323
RatecodeID               323
store_and_fwd_flag       323
PULocationID             323
DOLocationID             323
payment_type             323
fare_amount              323
extra                    323
mta_tax                  323
tip_amount               323
tolls_amount             323
improvement_surcharge    323
total_amount             323
fare_per_mile            323
trip_time                323
fare_per_minute            0
dtype: int64

In [64]:
df.fare_per_minute.fillna(0,inplace=True)

#### h) We created a new column, id_borough, for indetify the borough that the trip belong.

First, we loaded the NYC zones dataset.

In [108]:
df_zones = pd.read_csv('taxi+_zone_lookup.csv')

In [85]:
df_zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [86]:
df_zones.Borough.unique()

array(['EWR', 'Queens', 'Bronx', 'Manhattan', 'Staten Island', 'Brooklyn',
       'Unknown'], dtype=object)

We replaced the names of the boroughs for id's

In [90]:
df_zones.Borough.replace({"Bronx":0, "Brooklyn":1, "Manhattan":2, "Queens":3, "Staten Island":4, "EWR":5, "Unknown":6}, inplace=True)

We created a dictionary of zones with their respective borough_id, so then we can map

In [92]:
dic_zone_borough = {df_zones.LocationID[i] : df_zones.Borough[i] for i in range (0,len(df_zones))}

In [99]:
df['id_borough'] = df.PULocationID.map(dic_zone_borough)

#### i) We created a new column, id_time_borough, so then we can relationate with weather data

In [101]:
df['id_time_borough'] = df.tpep_pickup_datetime.dt.strftime('%Y%m%d%H') + df.id_borough.astype(str)

#### j) Trimming the data

In [103]:
df_final = df[(df.fare_amount > df.fare_amount.quantile(.05)) & (df.fare_amount < df.fare_amount.quantile(.95))]